In [1]:
import pandas as pd
from pathlib import Path
from collections import defaultdict
from itertools import product

In [ ]:
# Configuration - Input the Data folder here, Mark the "Control File", i.e. the One that as per domain knowledge has the Composite-Key, and mention the Key in "CNTKEY_COL"  to be splitted and traversed through the dataset
DATA_FOLDER      = Path(r"C:\Users\shobh\OneDrive\Desktop\Analytics Project\Dataset\test_data")
CONTROL_FILENAME = "ODS_CAMA_PRD.CNTRL.0.csv"
CNTKEY_COL       = "cntkey"
MIN_SPLIT_LEN    = 2    # minimum segment length for fallback
MAX_SPLIT_LEN    = 4    # maximum segment length for fallback
MIN_PARTS        = 2    # minimum number of segments per key
MAX_PARTS        = 4    # maximum number of segments per key

In [ ]:
def normalize_series(s: pd.Series) -> pd.Series:
    """Trim whitespace, drop blanks, lowercase."""
    s = s.fillna("").astype(str).str.strip()
    return s[s != ""].str.lower()

def load_control_keys(folder: Path) -> pd.Index:
    """Load unique CNTKEYs from the control table."""
    df = pd.read_csv(folder / CONTROL_FILENAME, dtype=str, low_memory=False)
    cols = {c.lower(): c for c in df.columns}
    real = cols.get(CNTKEY_COL)
    if not real:
        raise KeyError(f"Column '{CNTKEY_COL}' not found in control table")
    return normalize_series(df[real]).unique()

def build_column_metadata(folder: Path) -> pd.DataFrame:
    """
    Build metadata (min/max value lengths, unique ratio) for every CSV column
    except the control table.
    """
    records = []
    control_stem = CONTROL_FILENAME.lower().replace(".csv", "")
    for path in folder.glob("*.csv"):
        stem = path.stem.lower()
        if stem == control_stem:
            continue
        df = pd.read_csv(path, dtype=str, low_memory=False)
        for col in df.columns:
            vals = normalize_series(df[col])
            lengths = vals.str.len()
            records.append({
                "table":       stem,
                "column":      col,
                "min_len":     int(lengths.min()) if not lengths.empty else 0,
                "max_len":     int(lengths.max()) if not lengths.empty else 0,
                "unique_ratio": vals.nunique() / len(vals) if len(vals) > 0 else 0
            })
    return pd.DataFrame(records)

def whitespace_split(key: str) -> list:
    """
    If key contains spaces, split on them; else return [].
    """
    parts = key.split()
    return parts if len(parts) > 1 else []

def fallback_splits(key: str) -> list:
    """
    Generate all splits into MIN_PARTS–MAX_PARTS segments,
    each segment length between MIN_SPLIT_LEN and MAX_SPLIT_LEN.
    """
    n = len(key)
    results = []
    def helper(i, path):
        if i == n:
            if MIN_PARTS <= len(path) <= MAX_PARTS:
                results.append(path.copy())
            return
        if len(path) >= MAX_PARTS:
            return
        for L in range(MIN_SPLIT_LEN, MAX_SPLIT_LEN + 1):
            j = i + L
            if j <= n:
                path.append(key[i:j])
                helper(j, path)
                path.pop()
    helper(0, [])
    return results


def score_column_pairs(folder: Path, metadata: pd.DataFrame) -> pd.DataFrame:
    # 1) Load all tables
    tables = {}
    control_stem = CONTROL_FILENAME.lower().replace(".csv", "")
    for path in folder.glob("*.csv"):
        stem = path.stem.lower()
        df = pd.read_csv(path, dtype=str, low_memory=False).apply(normalize_series)
        tables[stem] = df

    # 2) Metadata lookup
    meta_map = {(row.table, row.column): row for row in metadata.itertuples()}

    # 3) Load CNTKEYs
    keys = load_control_keys(folder)

    # 4) Accumulate scores
    acc = defaultdict(lambda: {"score": 0, "examples": set()})
    for key in keys:
        parts = whitespace_split(key)
        splits = [parts] if parts else fallback_splits(key)

        for split in splits:
            # find candidate columns for each segment
            candidates = []
            for seg in split:
                L = len(seg)
                cand = []
                for (tbl, col), meta in meta_map.items():
                    if meta.min_len <= L <= meta.max_len:
                        cnt = (tables[tbl][col] == seg).sum()
                        if cnt > 0:
                            cand.append((tbl, col, cnt))
                candidates.append(cand)

            # score every combination
            for combo in product(*candidates):
                cols = tuple((t, c) for t, c, _ in combo)
                score = sum(cnt for _, _, cnt in combo)
                entry = acc[cols]
                entry["score"] += score
                if len(entry["examples"]) < 5:
                    entry["examples"].add(key)

    # 5) Build result DataFrame
    rows = []
    for cols, info in acc.items():
        rows.append({
            "column_pair":   cols,
            "total_score":   info["score"],
            "example_keys":  ", ".join(info["examples"])
        })
    result = pd.DataFrame(rows)
    return result.sort_values("total_score", ascending=False).reset_index(drop=True)

In [ ]:
if __name__ == "__main__":
    # Build metadata and score splits
    metadata = build_column_metadata(DATA_FOLDER)
    result   = score_column_pairs(DATA_FOLDER, metadata)

    # Show top candidates
    print("Top candidate column-pairs:")
    print(result.head(10).to_string(index=False))

    # Optionally save to CSV
    out = "cntkey_split_scores2_new.csv"
    result.to_csv(out, index=False)
    print(f"\nResults saved to {out}")

Top candidate column-pairs:
                                                     column_pair  total_score                                   example_keys
((ods_cama_prd.wordm.0, WOSREA), (ods_cama_prd.wocon.0, WCSERV))       279369   06 tdp, 10 pdha3, 10 hgob1, 10 pdmv1, 06 dks
((ods_cama_prd.woact.0, KMDSEQ), (ods_cama_prd.wocon.0, WCSERV))       263236 10 pdha3, 10 hgob1, 10 dkw, 10 pdmv1, 10 dthb1
((ods_cama_prd.wordm.0, WOSREA), (ods_cama_prd.ratem.0, SESRCD))       233321 10 pdha3, 10 hgob1, 10 pdmv1, 10 dthb1, 06 dks
 ((ods_cama_prd.wordm.0, WOSREA), (ods_cama_prd.wordd.0, WDSRC))       222531 10 pdha3, 10 hgob1, 10 pdmv1, 10 dthb1, 06 dks
 ((ods_cama_prd.woact.0, KMDSEQ), (ods_cama_prd.wordd.0, WDSRC))       222024 10 pdha3, 10 hgob1, 10 dkw, 10 pdmv1, 10 dthb1
((ods_cama_prd.woact.0, KMDSEQ), (ods_cama_prd.ratem.0, SESRCD))       219255 10 pdha3, 10 hgob1, 10 dkw, 10 pdmv1, 10 dthb1
 ((ods_cama_prd.woact.0, KMDLY), (ods_cama_prd.wocon.0, WCSERV))       197307 10 pdha3, 10 hgob1,